# Bitcoin price forecasting using ARMA

##  Objectives


* perform EDA on time series data
* analyze the auto correlation and partial auto correlation plots
* implement the ARMA model and forecast the bit coin price

## Dataset



Bitcoin is a digital currency created in January 2009. It follows the ideas set out in a whitepaper by the mysterious and pseudonymous Satoshi Nakamoto. The identity of the person or persons who created the technology is still a mystery. Bitcoin offers the promise of lower transaction fees than traditional online payment mechanisms and, unlike government-issued currencies, it is operated by a decentralized authority.

Data Description
This dataset provides the history of daily prices of Bitcoin. The data starts from 17-Sep-2014 and is updated till 09-July-2021. All the column descriptions are provided below.

* Date: Day/Month/Year
* Open: Price from the first transaction of a trading day
* High: Maximum price in a trading day
* Low: Minimum price in a trading day
* Close: Price from the last transaction of a trading day
* Adj Close: Closing price adjusted to reflect the value after accounting for any corporate actions
* Volume: Number of units traded in a day

## Problem Statement

Perform EDA and forecast the Bitcoin price using ARMA model on timeseries (bitcoin) data.

### Import required Packages

In [ ]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import itertools

### Load the data and perform EDA

Refer to this ['Bitcoin dataset EDA'](https://medium.com/@hamzaahmad86/exploratory-data-analysis-of-cryptocurrency-historical-data-d8ec719641e7)

In [ ]:
# reading the .csv file
path = "/content/BTC.csv"
df = pd.read_csv(path)
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',drop=True, inplace=True)
df.fillna(0, inplace=True)

In [ ]:
df.head()

In [ ]:
# plotting multiple variables
df.plot(subplots=True)
plt.show()

In [ ]:
pd.plotting.lag_plot(df['Close'], lag=10)

#### Analyze the correlation (heatmap) of all the features

In [ ]:
import seaborn as sns
g = sns.pairplot(df[df.columns])

By this we can observe the correlation between the features.

We can also find the values of correlation by using pearson correlation matrix.

In [ ]:
aq_pear_corr = df[df.columns].corr(method='pearson')
aq_pear_corr

In [ ]:
sns.heatmap(aq_pear_corr,annot=True)

Almost all the features posses same information, so we can select any of the given variables

In [ ]:
df['Close'].plot(figsize=(15,6))
plt.show()

### Testing the Stationarity using Augmented Dicky Fuller Test

The Augmented Dicky Fuller test is a type of statistical test called a unit root test.

[tsa.stattools.adfuller](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html)

In [ ]:
#Perform Dickey Fuller test
result = adfuller(df['Close'])
print('ADF Stastistic: %f'%result[0])
print('p-value: %f'%result[1])
pvalue = result[1]
for key,value in result[4].items():
  if result[0]>value:
    print("The graph is non stationery")
    break
  else:
    print("The graph is stationary")
    break;
print('Critical values:')
for key,value in result[4].items():
    print('\t%s: %.3f ' % (key, value))

### Identify the trends and seasonality from the given time series data

* Apply seasonal decompose and plot the results
* Check the stationarity of data using rolling mean and rolling standard deviation.
* Make the time series data stationary
  * Apply a log transformation to reduce the variance of the series
  * Eliminate the Trend and Seasonality by Differencing

**Note:** Ensure timeseries without NaN, inf, -inf values, Replace with 0 if found.

Read more about stationarity of a timeseries in the following [link](https://machinelearningmastery.com/time-series-data-stationary-python/)

In [ ]:
ts = df['Close']
# ETS Decomposition
result = seasonal_decompose(ts)
# ETS plot
result.plot();

In [ ]:
# ETS Decomposition
plt.figure(figsize=(30,10))
result = seasonal_decompose(ts[:1000])
# ETS plot
result.plot();

In [ ]:
# Let’s create a function to run the test which determines whether a given time series is stationary
def get_stationarity(timeseries):
    # Rolling statistics
    rolling_mean = timeseries.rolling(window=24).mean()
    rolling_std = timeseries.rolling(window=24).std()

    # Rolling statistics plot
    plt.figure(figsize=(15,5))
    original = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(rolling_mean, color='red', label='Rolling Mean')
    std = plt.plot(rolling_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

In [ ]:
get_stationarity(df['Close'])

In [ ]:
# ESTIMATING THE TREND
# Applying a log transformation is a way to reduce the variance of the series
df_log = np.log(df['Close'])
plt.plot(df_log);

In [ ]:
# replacing -inf values with 0
print(df_log.min())
df_log.replace(-np.inf, 0,inplace=True)
df_log.min()

Eliminating the Trend and Seasonality by **Differencing** (taking the difference with a particular time lag)

In [ ]:
shift_df = pd.concat([df_log, df_log.shift(7)],axis=1)
shift_df.columns = ['Actual','Forecasted']
shift_df.head()

In [ ]:
df_log_shift = shift_df['Actual'] - shift_df['Forecasted']
df_log_shift.dropna(inplace=True)
get_stationarity(df_log_shift)

In [ ]:
time_series = df_log#['Close']
time_series = time_series.diff(7).dropna()

### Test the Stationarity using Augmented Dicky Fuller Test

Verify the stationarity post differencing, using ADF

 [tsa.stattools.adfuller](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html)

In [ ]:
#Perform Dickey Fuller test
result=adfuller(time_series)
print('ADF Stastistic: %f'%result[0])
pvalue=result[1]
print('p-value: %f'%pvalue)

for key,value in result[4].items():
      if result[0]>value:
        print("The graph is non stationery")
        break
      else:
        print("The graph is stationary")
        break;
print('Critical values:')
for key,value in result[4].items():
    print('\t%s: %.3f ' % (key, value))

### Auto Correlation Plot
Autocorrelation refers to the degree of correlation between the values of the same variables across different observations in the data.  The concept of autocorrelation is most often discussed in the context of time series data in which observations occur at different points in time.

* Plot the auto correlation function (ACF and PACF)
* Analyse ACF and PACF plots and define AR (p) and MA(q) terms

In [ ]:
# let us plot acf and pacf graphs
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

plt.figure(figsize = (20,10))
plt.subplot(211)
plot_acf(time_series, ax=plt.gca(), lags = 60)
plt.subplot(212)
plot_pacf(time_series, ax=plt.gca(), lags = 60)
plt.show()

**Analysing ACF and PACF plots for Identifying the number of AR and MA terms (p and q)**

- The lag beyond which the PACF cuts off is the indicated number of AR terms (p).
- The lag beyond which the ACF cuts off is the indicated number of MA terms (q).

### Train the AR model

In [ ]:
regr = sm.tsa.AR(time_series).fit(lag=7)
fore = regr.predict()

time_series.plot()
fore.plot(figsize=(30,10))
plt.show()

### ARMA Model

* Fit the ARMA model on train data and forecast the test data

In [ ]:
# ARMA Method - 1
from math import sqrt
from sklearn.metrics import mean_squared_error

# ARMA model
model = ARMA(time_series, order=(4, 1))
model_fit = model.fit()
print(model_fit.aic)

predictions = model_fit.predict()
error = sqrt(mean_squared_error(time_series, predictions))
print('RMSE value: %.3f' % error)

In [ ]:
plt.figure(figsize=(27,7))
plt.plot(time_series.index, time_series, color='red', label='Actual Test Data')
plt.plot(time_series.index, predictions, color='green', linestyle='dashed', label='Predicted Data')
plt.legend();
plt.show()

In [ ]:
future = model_fit.predict(start=len(time_series),end=len(time_series)+7)
plt.title("7 Days forecast")
plt.plot(future.index, future, color='blue', linestyle='dashed')
plt.show()

### ARMA model in iterative process

In [ ]:
# split the data into train and test
train_ar = time_series[:int(len(time_series)*0.9)]
test_ar = time_series[int(len(time_series)*0.9):]

In [ ]:
len(train_ar), len(test_ar)

In [ ]:
# ARMA Method - 2
history = [x for x in train_ar.values]
predictions = list()

for t in range(len(test_ar)):
    model = ARMA(history, order=(4,1) )
    model_fit = model.fit()
    output = model_fit.forecast() # one-step forecast
    yhat = output[0]
    predictions.append(yhat)
    obs = test_ar.values[t]
    history.append(obs)
    print(model_fit.aic)

#### Plot the predictions for timeseries data

In [ ]:
plt.figure(figsize=(17,7))
plt.plot(train_ar.index,train_ar,label="Train data")
plt.plot(test_ar.index, test_ar, color='red', label='Test Data')
plt.plot(test_ar.index, predictions, color='green', linestyle='dashed', label='Predicted Data')
plt.legend();

### Report Analysis

* Discuss how sudden effects of bitcoin price effects the model parameters
* State your observations about the trend and seasonality of the timeseries data
* Discuss how you selected ARMA (p, d and q) terms.
* Interpret the AIC value obtained for the ARMA model

Internal Reference: https://towardsdatascience.com/bitcoin-price-prediction-using-time-series-forecasting-9f468f7174d3

#### ARIMA model

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(time_series,order=(4,0,1))
model_fit = model.fit()
model_fit.summary()

In [ ]:
print(model_fit.aic)

predictions = model_fit.predict()
error = sqrt(mean_squared_error(time_series, predictions))
print('RMSE value: %.3f' % error)